In [1]:
%load_ext autoreload
%autoreload 2
import json
from datetime import date

import polars as pl

from property_models.constants import DATA_DIR, POSTCODE_CSV_FILE, PropertyType, RecordType
from property_models.models import Address, HistoricalPrice, PropertyInfo

In [2]:
PropertyType.APARTMENT.SIXTIES_BRICK.value

('apartment', 'sixties_brick')

In [10]:
property_info = PropertyInfo(
    property_type=PropertyType.parse(PropertyType.APARTMENT.SIXTIES_BRICK),
    address=Address.parse("80 ROSEBERRY STREET, NORTH MELBOURNE, VIC 3032", country="australia"),
    condition=None,
    beds=10,
    cars=10,
    baths=10,
    floors=10,
    land_size_m2=100.3,
    property_size_m2=304.4,
    date_of_construction=date(2000, 1, 1),
)
property_info

PropertyInfo(address=Address(unit_number=None, street_number=80, street_name='ROSEBERRY STREET', suburb='NORTH MELBOURNE', post_code=3032, state='VIC', country='australia'), beds=10, baths=10, cars=10, property_size_m2=304.4, land_size_m2=100.3, condition=None, property_type=('apartment', 'sixties_brick'), date_of_construction=datetime.date(2000, 1, 1), floors=10)

In [47]:
pl.DataFrame([property_info]).select(pl.col("address").struct.unnest())
pl.DataFrame([property_info]).select(
    # pl.col('property_type')
    # .list.to_struct(fields = ["property_type_1","property_type_2"])
    # .struct['property_type_1']
    # .struct.unnest()
    property_type_general=pl.col("property_type").list[0],
    property_type_specific=pl.col("property_type").list[1],
)  # .rows(named= True)

property_type_general,property_type_specific
str,str
"""apartment""","""sixties_brick"""


In [5]:
(
    pl.DataFrame([property_info]).join(
        pl.DataFrame([property_info]).select("address", pl.lit("asdf")),
        on="address",
        how="inner",
    )
)

ComputeError: not yet implemented: Hash Inner Join between struct[7] and struct[7]

In [ ]:
data_json = property_info.model_dump()
data_string = json.dumps(data_json, default=str)

In [ ]:
data_loaded = json.loads(data_string)
data_loaded

{'address': {'unit_number': None,
  'street_number': 80,
  'street_name': 'ROSEBERRY STREET',
  'suburb': 'NORTH MELBOURNE',
  'post_code': 3032,
  'state': 'VIC',
  'country': 'australia'},
 'beds': 10,
 'baths': 10,
 'cars': 10,
 'property_size_m2': 304.4,
 'land_size_m2': 100.3,
 'condition': None,
 'property_type': ['apartment', 'sixties_brick'],
 'date_of_construction': '2000-01-01',
 'floors': 10}

In [ ]:
property_info_reloaded = PropertyInfo(
    address=Address(**data_loaded.pop("address")),
    property_type=PropertyType(data_loaded.pop("property_type")),
    date_of_construction=date.fromisoformat(data_loaded.pop("date_of_construction")),
    **data_loaded,
)
property_info_reloaded

In [ ]:
property_info_reloaded == property_info

True

In [ ]:
data_loaded = json.loads(data_string)
data_loaded.pop("bed")

property_info_reloaded = PropertyInfo(
    address=Address(**data_loaded.pop("address")),
    property_type=PropertyType(data_loaded.pop("property_type")),
    date_of_construction=date.fromisoformat(data_loaded.pop("date_of_construction")),
    beds=None,
    **data_loaded,
)
property_info_reloaded

KeyError: 'bed'

In [ ]:
property_info_reloaded == property_info

False